In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

### 문제 1
호주 멜버른의 주택 가격 데이터셋 melb_data.csv으로 주택 가격을 예측하고,   
각종 평가지표를 사용하여 결정트리 모델을 평가하세요.

#### Data Description 

* `Rooms`: 방 개수

* `Price`: 가격(단위: $ 달러)

* `Type`: 주택 유형   
br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential.

* `Distance`: Distance from CBD(Central Business District : 도심 지역)

* `Bedroom2` : Scraped # of Bedrooms (from different source) (주요 침실 개수)

* `Bathroom`: Number of Bathrooms

* `Car`: Number of carspots

* `Landsize`: Land Size

* `BuildingArea`: Building Size

* `CouncilArea`: Governing council for the area


우선 다음 코드들을 실행하여 melb_data변수명으로 데이터를 로드하고   
결정트리 분석에 필요한 컬럼만 남기고 결손값을 처리합니다.
      
* feature 데이터 = [Rooms, Type, Distance, Bedroom2, Bathroom, Car, Landsize, BuildingArea, CouncilArea]   
* label 데이터 = Price      
   
(Price 값이 양적 자료이기 때문에 분류보단 회귀방식의 모델이 더 적합하지만, 결정트리를 사용하기 위해 편의상 Price값을 사분위수에 따라 범주형 자료로 변환하여 사용합니다.)

In [13]:
#(문제 아님)
file_path = '../data/melb_data.csv'
drop_columns=['Date','YearBuilt','Suburb','SellerG','Postcode','Address','Method','Lattitude','Longtitude','Propertycount']
melb_data = pd.read_csv(file_path).drop(columns=drop_columns,axis=1)
melb_data['CouncilArea'] = melb_data['CouncilArea'].fillna('Moreland')
melb_data['BuildingArea'] = melb_data['BuildingArea'].fillna(melb_data['BuildingArea'].mean())
melb_data['Car'] = melb_data['Car'].fillna(melb_data['Car'].median())

In [3]:
melb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rooms         13580 non-null  int64  
 1   Type          13580 non-null  object 
 2   Price         13580 non-null  float64
 3   Distance      13580 non-null  float64
 4   Bedroom2      13580 non-null  float64
 5   Bathroom      13580 non-null  float64
 6   Car           13580 non-null  float64
 7   Landsize      13580 non-null  float64
 8   BuildingArea  13580 non-null  float64
 9   CouncilArea   13580 non-null  object 
 10  Regionname    13580 non-null  object 
dtypes: float64(7), int64(1), object(3)
memory usage: 1.1+ MB


LabelEncoder를 사용하여 인코딩을 합니다.

In [4]:
#(문제 아님)
categorical_features = [ 'Type', 'CouncilArea','Regionname']
for col in categorical_features:
    le = LabelEncoder()
    melb_data[col] = le.fit_transform(melb_data[col])

### 1-1 (5점) - 전처리
label 데이터(target)로 Price 를 사용할 것입니다.   
그러나 양적 자료인 Price를 결정트리에 알맞게 범주형 자료로 변형하기 위해서 사분위수(quartile)와 2차세션 복습과제 때 학습했던 Tukey Fences 개념을 활용할 것입니다.   
아래 규칙과 같이 Price값을 바꾸세요.
   
   <span style='color:blue'>[ 규칙 ]</span>
* 0 이상, Q1 미만 : '0' 
* Q1 이상, Q2 미만 : '1' 
* Q2 이상, Q3 미만 : '2' 
* Q3 이상 : '3' 
      
다음은 커스텀 함수 price_to_level()입니다. 위의 규칙에 맞게 빈칸을 알맞게 채우세요.

In [ ]:
def price_to_level(prices): # prices : pd.Series객체로 전달해주세요
    # 사분위수(Q1,Q2,Q3) 구하기
    
    # 문제) 위의 규칙에 맞게 Price값에 따라 레벨로 변환하도록 코드를 작성하세요
    p_levels = []
    
    # Price_Level 데이터를 반환하세요.
    return p_levels    

# price_to_level() 커스텀 함수를 사용하여 Price컬럼값을 변경합니다.
melb_data['Price'] = price_to_level(melb_data['Price'])
# 문제) Price 컬럼명을 PriceLevel로 변경하세요. (힌트: pandas의 rename() 사용)


In [ ]:
# 답)
'''
def price_to_level(prices): # prices : pd.Series객체로 전달해주세요
    # 사분위수와 LF, UF
    Q1,Q2,Q3 = np.percentile(prices,[25,50,75])
    # 문제) 위의 규칙에 맞게 Price값에 따라 레벨로 변환하도록 코드를 작성하세요
    intervals = [(0,Q1),(Q1,Q2),(Q2,Q3),(Q3,prices.max()+1)]
    p_levels = []
    for p in prices:
        for i,interval in enumerate(intervals):
            if interval[0] <= p < interval[1]:
                p_levels.append(f'{i}')
    # Price_Level 데이터를 반환하세요.
    return p_levels    

# price_to_level() 커스텀 함수를 사용하여 Price컬럼값을 변경합니다.
melb_data['Price'] = price_to_level(melb_data['Price'])
# 문제) Price 컬럼명을 PriceLevel로 변경하세요. (힌트: pandas의 rename() 사용)
melb_data = melb_data.rename(columns={'Price':'PriceLevel'})
'''

아래 코드를 실행하여 데이터 변환이 제대로 이루어졌는지 확인하세요.

In [6]:
#(문제아님)
### 간단하게 price_to_level() 커스텀 함수를 확인하세요 ###
print(melb_data.loc[8,'PriceLevel'],melb_data.loc[3,'PriceLevel'],
      melb_data.loc[1,'PriceLevel'],melb_data.loc[0,'PriceLevel'])
### 출력 결과가 0 1 2 3 이면, 커스텀 함수를 올바르게 작성한 것입니다. ###

0 1 2 3


### 1-2 (5점) - 정확도
melb_data 를 train_test_split() 함수로 학습/테스트용 데이터로 분리하고, 간단하게 DecisionTreeClassifier 를 사용하여 모델을 학습하고 PriceLevel을 예측해보세요. 그리고 예측 결과의 정확도(accuracy)를 평가해서 소수점 5번째 자리에서 반올림하여 출력해보세요.   
<span style='color:green'>(DecisionTreeClassfier 생성자 함수와 train_test_split의 random_state 파라미터값은 모두 10을 전달해주세요, test_size는 0.25로 설정해주세요.)</span>
* feature 데이터 : PriceLevel을 제외한 나머지 컬럼   
* label 데이터 : PriceLevel

In [ ]:
# train_test_split()으로 데이터를 분리하세요.


# 결정트리 객체를 생성하고 학습, 예측을 수행하세요. (예측값은 'pred' 변수명으로 저장하세요.)


# 예측 결과를 평가하고 평가 결과를 위의 조건대로 반올림하여 출력하세요


In [7]:
# 답)
'''
# train_test_split()으로 데이터를 분리하세요.
features = melb_data.drop('PriceLevel',axis=1)
label = melb_data['PriceLevel']
X_train,X_test,y_train,y_test = train_test_split(features,label,test_size=0.25,random_state=10)

# 결정트리 객체를 생성하고 학습, 예측을 수행하세요. (예측값은 'pred' 변수명으로 저장하세요.)
dt_clf = DecisionTreeClassifier(random_state=10)
dt_clf.fit(X_train,y_train)
pred = dt_clf.predict(X_test)

# 예측 결과를 평가하고 평가 결과를 위의 조건대로 반올림하여 출력하세요
np.round(accuracy_score(y_test,pred),4)
'''

0.6236

### 1-3 (30점) - 정밀도와 재현율
위의 예측값(pred)과 실제값(y_test)을 토대로 confusion matrix를 구현해보세요.   (다중 분류일 때의 confusion matrix를 학습해봅시다)   
다음과 같은 모양으로 DataFrame객체를 생성해서 출력해보세요.   
   (개수는 해당 경우의 수를 의미)
|          | 0 | 1 | 2 | 3 |
|----------|----------|----------|----------|----------|
| 0 | 개수 | 개수 |개수  |개수  |
| 1 | 개수 | 개수 | 개수 | 개수 |
| 2 | 개수 | 개수 | 개수 | 개수 |
| 3 | 개수 | 개수 | 개수 | 개수 |

**a)**
sklearn.metrics의 confusion_matrix() 함수로 생성한 오차행렬을 cm 변수에 저장하고,
이를 DataFrame객체로 변환해 위의 예시처럼 예쁘게 출력하세요.   
**(3점)**

In [ ]:
classes = ['0', '1', '2', '3'] # 레이블 클래스
# 변수 cm에 confusion_matrix를 저장하세요

# pandas DataFrame으로 변환하여 예쁘게 출력하세요


In [8]:
# 답)
'''
classes = ['0', '1', '2', '3'] # 레이블 클래스
# 변수 cm에 confusion_matrix를 저장하세요
cm = (confusion_matrix(y_test,pred))
# pandas DataFrame으로 변환하여 예쁘게 출력하세요
pd.DataFrame(cm,index=classes,columns=classes)
'''

,0,1,2,3
0,649,183,22,4
1,175,434,186,45
2,32,186,450,196
3,8,31,210,584


####  직접 재현율(recall)과 정밀도(precision)를 구해서 확인해보세요
힌트: 다중분류의 confusion matrix를 학습하고 macro-averaging과 micro-averaging을 학습하세요.

**b)** macro-averaging(macro-mean) 방식을 사용하여 macro_precision과 macro_recall 값을 구하고 출력하세요.   
(반복문을 사용하여 해결하세요, 변수명도 각각 macro_precision과 macro_recall을 사용하세요.)      
**(10점)**

In [9]:
# 답)
macro_precisions = []
TPS = []
FPS = []
for c in range(cm.shape[0]):
    TP = cm[c][c]
    FP = 0
    for r in range(cm.shape[0]):
        if c == r :
            continue
        FP += cm[r][c]
    macro_precisions.append(TP/(TP+FP))
    TPS.append(TP)
    FPS.append(FP)
macro_precision = np.mean(macro_precisions)
print('macro_precision:',macro_precision)
macro_recalls = []
FNS = []
for idx, row in enumerate(cm):
    TP = row[idx]
    rlist = list(row)
    rlist.pop(idx)
    FN = sum(rlist)
    macro_recalls.append(TP/(TP+FN))
    FNS.append(FN)
macro_recall = np.mean(macro_recalls)
print('macro_recall:',macro_recall) 

macro_precision: 0.623609372215409
macro_recall: 0.6237476721457814


**c)** micro-averaging 방식을 사용하여 micro_precision과 micro_recall 값을 구하고 출력하세요.   
(변수명도 각각 micro_precision과 micro_recall을 사용하세요.)   
(힌트: 위의 반복문에서 미리 TP,FP,FN들을 저장하여 여기선 계산만 하세요.)   
**(10점)**

In [10]:
# 답)
micro_precision = sum(TPS)/(sum(TPS)+sum(FPS))
micro_recall = sum(TPS)/(sum(TPS)+sum(FNS))
print('micro_precision:',micro_precision)
print('micro_recall:',micro_recall)

micro_precision: 0.6235640648011782
micro_recall: 0.6235640648011782


**d)** sklearn.metrics의 classification_report() 함수를 사용하여 구한 평가지표 결과를 clf_report 변수에 저장하세요.(힌트: output_dict 파라미터를 True로 설정)   
그리고 해당 변수를 DataFrame 객체로 변환하여 출력하세요.   
그리고 위에서 직접 구한 macro_precision, macro_recall, micro_precision, micro_recall값이 clf_report의 내용과 같은지 소수점 4번째 자리까지 비교하여 출력하세요.
* 출력용 코드 예시   
<code>print('[Macro_P] my_score:{:.4f}, report_score:{:.4f}'.format(macro_precision,report_mac_precision))</code>   
<code>print('[Macro_R] my_score:{:.4f}, report_score:{:.4f}'.format(macro_recall,report_mac_recall))</code>   
<code>print('[Micro_P] my_score:{:.4f}, report_score:{:.4f}'.format(micro_precision,report_mic_precision))</code>   
<code>print('[Micro_R] my_score:{:.4f}, report_score:{:.4f}'.format(micro_recall,report_mic_recall))</code>
      
(힌트: classification_report를 출력해보면 micro avg 데이터가 없다! 그러나 micro avg의 수식을 잘 살펴보면 어떤 평가지표와 수식의 의미가 똑같다는 사실을 알 수 있다.)   
**(7점)**

In [ ]:
# classification_report를 변수에 저장하고 DataFrame으로 변환하여 출력하세요.


In [11]:
# 답)
clf_report = classification_report(y_test,pred,labels=classes,target_names=classes,output_dict=True)
clf_report_df = pd.DataFrame(clf_report)
clf_report_df

,0,1,2,3,accuracy,macro avg,weighted avg
precision,0.751157,0.520384,0.518433,0.704463,0.623564,0.623609,0.623375
recall,0.756410,0.516667,0.520833,0.701080,0.623564,0.623748,0.623564
f1-score,0.753775,0.518519,0.519630,0.702768,0.623564,0.623673,0.623464
support,858.000000,840.000000,864.000000,833.000000,0.623564,3395.000000,3395.000000


In [ ]:
# 위의 출력 코드 예시를 활용해서 각 평가 결과값들을 비교하여 출력하세요


In [12]:
# 답)
report_mac_precision,report_mac_recall = clf_report_df['macro avg'][[0,1]]
report_mic_precision,report_mic_recall = clf_report_df['accuracy'][[0,1]]
print('[Macro_P] my_score:{:.4f}, report_score:{:.4f}'.format(macro_precision,report_mac_precision))
print('[Macro_R] my_score:{:.4f}, report_score:{:.4f}'.format(macro_recall,report_mac_recall))
print('[Micro_P] my_score:{:.4f}, report_score:{:.4f}'.format(micro_precision,report_mic_precision))
print('[Micro_R] my_score:{:.4f}, report_score:{:.4f}'.format(micro_recall,report_mic_recall))

[Macro_P] my_score:0.6236, report_score:0.6236
[Macro_R] my_score:0.6237, report_score:0.6237
[Micro_P] my_score:0.6236, report_score:0.6236
[Micro_R] my_score:0.6236, report_score:0.6236
